In [1]:
import pandas as pd
import numpy as np
import re
import csv
import os
# import nltk
from numpy import triu
# nltk.download('words')
# from nltk.corpus import words
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases

In [2]:
data = pd.read_csv('new_data.csv',on_bad_lines='skip')
data

,Salts,Medicine_Name
0,amitriptyline 25 mg,amilift 25 sr tablet 10s
1,Amitriptyline(25.0 Mg),amilift sr 25mg tablets
2,Amitriptyline + Chlordiazepoxide,amilift dx tablets
3,Amitriptyline (12.5mg) + Chlordiazepoxide (5mg),amilift dx 12.5 mg 5 mg tablet
4,Amitriptyline(10.0 Mg),amilift 10mg tablets
...,...,...
517008,Cefuroxime (500mg),zeoroxim 500mg tablet
517009,Cefuroxime (500mg) + Clavulanic Acid (125mg),zeoroxim cv 500mg 125mg tablet
517010,Telmisartan (40mg),zicard 40mg tablet
517011,Deflazacort (6mg),zofla 6mg tablet


In [3]:
# filtered_data = data[data['Salts'].str.contains('Paracetamol', case=False, na=False)]
filtered_data = data.iloc[:,:]

In [4]:
filtered_data

,Salts,Medicine_Name
0,amitriptyline 25 mg,amilift 25 sr tablet 10s
1,Amitriptyline(25.0 Mg),amilift sr 25mg tablets
2,Amitriptyline + Chlordiazepoxide,amilift dx tablets
3,Amitriptyline (12.5mg) + Chlordiazepoxide (5mg),amilift dx 12.5 mg 5 mg tablet
4,Amitriptyline(10.0 Mg),amilift 10mg tablets
...,...,...
517008,Cefuroxime (500mg),zeoroxim 500mg tablet
517009,Cefuroxime (500mg) + Clavulanic Acid (125mg),zeoroxim cv 500mg 125mg tablet
517010,Telmisartan (40mg),zicard 40mg tablet
517011,Deflazacort (6mg),zofla 6mg tablet


In [5]:
filtered_data['Salts'] = filtered_data['Salts'].str.replace('('," ").str.replace(")","").str.replace("-"," ").str.replace("mg"," mg").str.replace("%"," %").str.replace(","," + ")

In [6]:
filtered_data['Salts'] = filtered_data['Salts'].str.lower()

In [7]:
def convert_integer_to_float(value):
    try:
        # Use regex to find the numeric part
        match = re.search(r"(\d+(\.\d+)?)", value)
        if match:
            numeric_part = match.group()
            # Check if the numeric part is an integer
            if numeric_part.isdigit():
                numeric_part = float(numeric_part)
                # Replace the integer part with its float representation
                value = re.sub(r"\d+", str(numeric_part), value, 1)
        return value
    except:
        return value

In [8]:
filtered_data['Salts']= filtered_data['Salts'].str.replace(r'^\s+|\s+$', '', regex=True).apply(convert_integer_to_float)

In [9]:
from tqdm.auto import tqdm
tqdm.pandas()
filtered_data['Salts'] = filtered_data['Salts'].str.strip().str.replace('mcg',' mcg').str.replace("mg"," mg").str.replace("ml","  ml").str.replace("w/w","").str.replace("   "," ").str.replace("  "," ").str.replace('% ','%')

In [10]:
filtered_data

,Salts,Medicine_Name
0,amitriptyline 25.0 mg,amilift 25 sr tablet 10s
1,amitriptyline 25.0 mg,amilift sr 25mg tablets
2,amitriptyline + chlordiazepoxide,amilift dx tablets
3,amitriptyline 12.5 mg + chlordiazepoxide 5 mg,amilift dx 12.5 mg 5 mg tablet
4,amitriptyline 10.0 mg,amilift 10mg tablets
...,...,...
517008,cefuroxime 500.0 mg,zeoroxim 500mg tablet
517009,cefuroxime 500.0 mg + clavulanic acid 125 mg,zeoroxim cv 500mg 125mg tablet
517010,telmisartan 40.0 mg,zicard 40mg tablet
517011,deflazacort 6.0 mg,zofla 6mg tablet


In [11]:
new_df = filtered_data.drop_duplicates(subset='Salts')
new_df = new_df.reset_index(drop=True)

In [12]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="phi3")

In [ ]:
# Function to embed each row
def get_embeddings(text):
    return embeddings.embed_query(text)

# Apply the function to each row in the 'salts' column
new_df['Embeddings'] = new_df['Salts'].apply(get_embeddings)

In [ ]:
len(new_df["Embeddings"][0])

3072

In [ ]:
# Convert df_train['Embeddings'] Pandas series to a np.array of float32
X = np.array(new_df['Embeddings'].to_list(), dtype=np.float32)
X.shape

(1807, 3072)

In [ ]:
import umap.umap_ as umap
reducer = umap.UMAP(
    n_components=3,
    n_neighbors=15,
    n_epochs=200,
    min_dist=0.1,
    local_connectivity=1,
    spread= 1.0
)
embedding = reducer.fit_transform(np.array(X))

In [ ]:
# from sklearn.manifold import TSNE
# model = TSNE(n_components=3, perplexity=1400.0,
#           early_exaggeration=10.0,
#             learning_rate='auto',
#               n_iter_without_progress=100,
#                 min_grad_norm=1e-07,
#                   metric='euclidean',
#                   metric_params=None, init='pca', verbose=0, random_state=None, method='exact', angle=0.5, n_jobs=None)
# embedding = model.fit_transform(np.array(X))

In [ ]:
embedding.shape

(1807, 3)

In [ ]:
final_list = list(zip(new_df['Medicine_Name'],new_df['Salts'],embedding))

In [ ]:
final_dataframe = pd.DataFrame(final_list,columns = ['Medicine_Name','Salts','vector'])

In [ ]:
final_dataframe

,Medicine_Name,Salts,vector
0,amilift 25 sr tablet 10s,amitriptyline 25.0 mg,"[7.6449814, 8.778024, 6.6443825]"
1,amilift dx tablets,amitriptyline + chlordiazepoxide,"[9.494292, 8.044148, 8.555421]"
2,amilift dx 12.5 mg 5 mg tablet,amitriptyline 12.5 mg + chlordiazepoxide 5 mg,"[5.133318, 9.246333, 7.1401486]"
3,amilift 10mg tablets,amitriptyline 10.0 mg,"[7.634665, 8.79363, 6.6378107]"
4,azibeta 500mg tablet,azithromycin 500.0 mg,"[6.5614185, 8.831736, 6.624531]"
...,...,...,...
1802,afdimoxin dt tablet 10s,amoxycillin 125.0 mg + lactobacillus 60 millio...,"[5.7385435, 7.6889534, 7.8825893]"
1803,afdimoxin tablet dt,amoxycillin 125.0 mg + lactobacillus 60million...,"[5.7229156, 7.691019, 7.8813543]"
1804,aanav-mox 625 lb tablet 10's,amoxycillin 500.0 mg + clavulanic acid 125 mg ...,"[5.757681, 7.6723814, 7.9111333]"
1805,azemo 1 eye drop,azithromycin 1.0 %w/v,"[7.904517, 6.6656575, 8.122447]"


In [ ]:
## implementing the quasi random sampling to select some anchor points.
total_samples = len(final_dataframe)
sample_size = len(final_dataframe)/3
period  = int(total_samples/sample_size)


In [ ]:
Vectors = final_dataframe['vector'].values

In [ ]:
anchors = []
anchor_indexes=[]
for i in range(0,len(final_dataframe)+1,period):
    try:
        anchors.append(final_dataframe['Medicine_Name'][i])
        anchor_indexes.append(i)
    except:
        pass
from numpy import dot
from numpy.linalg import norm

# cos_sim = dot(a, b)/(norm(a)*norm(b))
anchor_euc_scores = {}
for i in range(0, len(anchor_indexes)):
    print(str(i) + " done!")
    current_index = anchor_indexes[i]
    current_point = Vectors[current_index]
    euc_sim_scores = []
    for j in range(0, len(final_dataframe)):
        
        other_point = Vectors[j]

        # Check for zero norms
        norm_current = norm(current_point)
        norm_other = norm(other_point)

        if norm_current == 0 or norm_other == 0:
            euc_sim = 0  # or any other value you choose
        else:
            euc_sim = 1 / (1 + np.linalg.norm(norm_current - norm_other))

        euc_sim_scores.append([j, euc_sim * 100000])

    euc_sim_scores = sorted(euc_sim_scores , key=lambda k: [k[1], k[0]], reverse=True)
    anchor_euc_scores[anchor_indexes[i]] = euc_sim_scores[0:500]


0 done!
1 done!
2 done!
3 done!
4 done!
5 done!
6 done!
7 done!
8 done!
9 done!
10 done!
11 done!
12 done!
13 done!
14 done!
15 done!
16 done!
17 done!
18 done!
19 done!
20 done!
21 done!
22 done!
23 done!
24 done!
25 done!
26 done!
27 done!
28 done!
29 done!
30 done!
31 done!
32 done!
33 done!
34 done!
35 done!
36 done!
37 done!
38 done!
39 done!
40 done!
41 done!
42 done!
43 done!
44 done!
45 done!
46 done!
47 done!
48 done!
49 done!
50 done!
51 done!
52 done!
53 done!
54 done!
55 done!
56 done!
57 done!
58 done!
59 done!
60 done!
61 done!
62 done!
63 done!
64 done!
65 done!
66 done!
67 done!
68 done!
69 done!
70 done!
71 done!
72 done!
73 done!
74 done!
75 done!
76 done!
77 done!
78 done!
79 done!
80 done!
81 done!
82 done!
83 done!
84 done!
85 done!
86 done!
87 done!
88 done!
89 done!
90 done!
91 done!
92 done!
93 done!
94 done!
95 done!
96 done!
97 done!
98 done!
99 done!
100 done!
101 done!
102 done!
103 done!
104 done!
105 done!
106 done!
107 done!
108 done!
109 done!
110 done!


In [ ]:
with open(r'C:\Users\ayush\OneDrive\Documents\polyglot\PolyGlot\data\phi\meta-main.txt', 'w') as the_file:
    the_file.write('0\n')
    for i in range(0, len(anchors)):
        the_file.write(str(anchor_indexes[i]) + ' ' + anchors[i] + '\n')

with open(r'C:\Users\ayush\OneDrive\Documents\polyglot\PolyGlot\data\phi\full_data', 'w',encoding="utf-8") as the_file:
    for i in range(0, len(new_df['Medicine_Name'])):
        word = new_df['Medicine_Name'][i]
        if word in anchors:
            is_anchor = 1
        else:
            is_anchor = 0
        if word == 'Synonym'or word == 'Vitamin D3':
            pass
        else:
            
            the_file.write(word +' ----> '+new_df['Salts'][i] +'\n')
            try:
              the_file.write(str(Vectors[i][0]) + ' ' + str(Vectors[i][1]) + ' ' + str(Vectors[i][2]) + ' ' + str(is_anchor) + ' \n')
            except:
              the_file.write(str(Vectors[i]) +' ' + str(is_anchor) + ' \n')

folder_path  = r"C:\Users\ayush\OneDrive\Documents\polyglot\PolyGlot\data\phi"
for anchor_index in anchor_indexes:
    filename = os.path.join(folder_path, str(anchor_index) + '.txt')
    with open(filename, 'w') as the_file:
        for score in anchor_euc_scores[anchor_index]:
            # note that score is of the form: [index, cosine]
            the_file.write(str(score[0]) + ', ' + str(score[1]) + '\n')